# 공연 추천 시스템 실험 노트북

이 노트북에서는 KOPIS API를 활용한 공연 추천 시스템의 구현 과정을 단계별로 실험해보겠습니다.

## 1. 필요한 라이브러리 임포트

In [1]:
import os
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv

## 연동 테스트

In [6]:
# .env 파일 로드
load_dotenv()

# API 키를 .env 파일에서 가져오기
KOPIS_API_KEY = os.getenv('KOPIS_API_KEY')

if not KOPIS_API_KEY:
    raise ValueError("KOPIS_API_KEY가 설정되지 않았습니다. .env 파일을 확인해주세요.")

# 날짜 범위 설정
today = datetime.now()
start_date = today.strftime('%Y%m%d')  # YYYYMMDD 형식 (예: 20241230)
end_date = (today + timedelta(days=30)).strftime('%Y%m%d')  # YYYYMMDD 형식

# API 호출 테스트
base_url = "http://www.kopis.or.kr/openApi/restful/pblprfr"

# URL 인코딩을 확실히 하기 위해 requests의 params 사용
params = {
    'service': KOPIS_API_KEY,
    'stdate': start_date,
    'eddate': end_date,
    'rows': 10,
    'cpage': 1  # 페이지 번호도 필수 파라미터일 수 있음
}

response = requests.get(base_url, params=params)
print(f"API 응답 상태 코드: {response.status_code}")

# XML 응답 파싱
try:
    root = ET.fromstring(response.content)
    return_code = root.find('.//returncode')
    if return_code is not None:
        error_msg = root.find('.//errmsg').text
        print(f"API 에러: {error_msg}")
    else:
        print("\n=== 공연 목록 ===")
        for item in root.findall('.//db'):
            try:
                performance_id = item.find('mt20id').text
                name = item.find('prfnm').text
                print(f"공연ID: {performance_id}")
                print(f"공연명: {name}")
                print("-" * 30)
            except AttributeError as e:
                print(f"데이터 파싱 오류: {e}")
except ET.ParseError as e:
    print(f"XML 파싱 오류: {e}")

API 응답 상태 코드: 200

=== 공연 목록 ===
공연ID: PF256390
공연명: 바싸르 오케스트라 신년음악회 [김포]
------------------------------
공연ID: PF256375
공연명: 롤링 30주년 기념 공연, 행로난 콘서트: 심장의 언어
------------------------------
공연ID: PF256373
공연명: 프로젝트 우리 앨범 발매 콘서트
------------------------------
공연ID: PF256372
공연명: 패
------------------------------
공연ID: PF256371
공연명: 보고싶습니다 (앵콜)
------------------------------
공연ID: PF256369
공연명: 바리톤 차두식 독창회 [대전]
------------------------------
공연ID: PF256368
공연명: 먼데이프로젝트 시즌8, My Universe: 정효빈 단독 콘서트
------------------------------
공연ID: PF256366
공연명: 먼데이프로젝트 시즌8, 심아일랜드 단독 콘서트: Our Universe Party
------------------------------
공연ID: PF256365
공연명: 마주보는 집 [대전]
------------------------------
공연ID: PF256363
공연명: 78 LIVE, 손을 모아
------------------------------


## 파싱 테스트

In [7]:
def parse_performance_list(xml_text):
    root = ET.fromstring(xml_text)
    performances = []
    
    for db in root.findall('.//db'):
        performance = {
            'mt20id': db.find('mt20id').text if db.find('mt20id') is not None else None,
            'prfnm': db.find('prfnm').text if db.find('prfnm') is not None else None,
            'genrenm': db.find('genrenm').text if db.find('genrenm') is not None else None,
            'prfpdfrom': db.find('prfpdfrom').text if db.find('prfpdfrom') is not None else None,
            'prfpdto': db.find('prfpdto').text if db.find('prfpdto') is not None else None,
            'fcltynm': db.find('fcltynm').text if db.find('fcltynm') is not None else None,
            'poster': db.find('poster').text if db.find('poster') is not None else None
        }
        performances.append(performance)
    
    return pd.DataFrame(performances)

# 파싱 테스트
df = parse_performance_list(response.text)
print("파싱된 데이터프레임 미리보기:")
display(df.head())

파싱된 데이터프레임 미리보기:


,mt20id,prfnm,genrenm,prfpdfrom,prfpdto,fcltynm,poster
0,PF256390,바싸르 오케스트라 신년음악회 [김포],서양음악(클래식),2025.01.04,2025.01.04,김포아트홀,http://www.kopis.or.kr/upload/pfmPoster/PF_PF2...
1,PF256375,"롤링 30주년 기념 공연, 행로난 콘서트: 심장의 언어",대중음악,2025.01.24,2025.01.24,롤링홀,http://www.kopis.or.kr/upload/pfmPoster/PF_PF2...
2,PF256373,프로젝트 우리 앨범 발매 콘서트,대중음악,2025.01.04,2025.01.04,플렉스홀 3호점,http://www.kopis.or.kr/upload/pfmPoster/PF_PF2...
3,PF256372,패,연극,2025.01.16,2025.01.19,소극장 플랫폼74,http://www.kopis.or.kr/upload/pfmPoster/PF_PF2...
4,PF256371,보고싶습니다 (앵콜),연극,2025.01.09,2025.01.19,후암스테이지,http://www.kopis.or.kr/upload/pfmPoster/PF_PF2...


## 2. 장르 코드 정의

In [2]:
# 올바른 장르 코드 정의
GENRE_CODES = {
    'AAAA': '연극',
    'BBBC': '무용(서양/한국무용)',
    'BBBE': '대중무용',
    'CCCA': '서양음악(클래식)',
    'CCCC': '한국음악(국악)',
    'CCCD': '대중음악',
    'EEEA': '복합',
    'EEEB': '서커스/마술',
    'GGGA': '뮤지컬'
}

In [12]:
def analyze_plot_ratio_by_genre():
    """6개월 범위의 장르별 줄거리 포함 비율 분석"""
    
    # 날짜 범위 설정 (6개월)
    today = datetime.now()
    start_date = today.strftime('%Y%m%d')
    end_date = (today + timedelta(days=90)).strftime('%Y%m%d')
    
    results = {}
    total_performances = 0
    total_with_plot = 0
    
    print("=== 장르별 줄거리 포함 비율 분석 ===\n")
    print("분석 기간:", datetime.strptime(start_date, '%Y%m%d').strftime('%Y-%m-%d'),
          "~", datetime.strptime(end_date, '%Y%m%d').strftime('%Y-%m-%d'))
    print("\n각 장르별 분석 중...\n")
    
    for code, genre_name in GENRE_CODES.items():
        # 공연 목록 조회
        base_url = "http://www.kopis.or.kr/openApi/restful/pblprfr"
        params = {
            'service': KOPIS_API_KEY,
            'stdate': start_date,
            'eddate': end_date,
            'rows': 100,  # 최대한 많은 데이터 수집
            'cpage': 1,
            'shcate': code
        }
        
        try:
            response = requests.get(base_url, params=params)
            if response.status_code != 200:
                print(f"{genre_name} 데이터 조회 실패")
                continue
                
            root = ET.fromstring(response.text)
            performances = root.findall('.//db')
            
            if not performances:
                print(f"{genre_name}: 해당 기간 공연 없음")
                continue
            
            genre_total = len(performances)
            plot_count = 0
            
            # 각 공연의 줄거리 정보 확인
            for perf in performances:
                mt20id = perf.find('mt20id').text
                detail_url = f"http://www.kopis.or.kr/openApi/restful/pblprfr/{mt20id}"
                detail_response = requests.get(detail_url, params={'service': KOPIS_API_KEY})
                
                if detail_response.status_code == 200:
                    detail_root = ET.fromstring(detail_response.text)
                    sty = detail_root.find('.//sty')
                    if sty is not None and sty.text and sty.text.strip():
                        plot_count += 1
            
            # 결과 저장
            percentage = (plot_count / genre_total * 100) if genre_total > 0 else 0
            results[genre_name] = {
                'total': genre_total,
                'with_plot': plot_count,
                'percentage': percentage
            }
            
            total_performances += genre_total
            total_with_plot += plot_count
            
            print(f"{genre_name}")
            print(f"  - 전체 공연 수: {genre_total}")
            print(f"  - 줄거리 포함 공연 수: {plot_count}")
            print(f"  - 줄거리 포함 비율: {percentage:.1f}%")
            print("-" * 50)
            
        except Exception as e:
            print(f"{genre_name} 분석 중 오류 발생: {str(e)}")
    
    # 전체 통계
    total_percentage = (total_with_plot / total_performances * 100) if total_performances > 0 else 0
    
    print("\n=== 전체 통계 요약 ===")
    print(f"전체 공연 수: {total_performances}")
    print(f"줄거리 포함 공연 수: {total_with_plot}")
    print(f"전체 줄거리 포함 비율: {total_percentage:.1f}%\n")
    
    # 장르별 순위 (줄거리 포함 비율 기준)
    print("장르별 줄거리 포함 비율 순위:")
    sorted_results = sorted(
        results.items(), 
        key=lambda x: x[1]['percentage'], 
        reverse=True
    )
    
    for genre, stats in sorted_results:
        print(f"{genre:15} - {stats['percentage']:.1f}% ({stats['with_plot']}/{stats['total']})")
    
    return results

# 분석 실행
if __name__ == "__main__":
    if not KOPIS_API_KEY:
        raise ValueError("KOPIS_API_KEY가 설정되지 않았습니다. .env 파일을 확인해주세요.")
        
    analysis_results = analyze_plot_ratio_by_genre()

=== 장르별 줄거리 포함 비율 분석 ===

분석 기간: 2024-12-30 ~ 2025-03-30

각 장르별 분석 중...

연극
  - 전체 공연 수: 100
  - 줄거리 포함 공연 수: 5
  - 줄거리 포함 비율: 5.0%
--------------------------------------------------
무용(서양/한국무용)
  - 전체 공연 수: 13
  - 줄거리 포함 공연 수: 1
  - 줄거리 포함 비율: 7.7%
--------------------------------------------------
대중무용: 해당 기간 공연 없음
서양음악(클래식)
  - 전체 공연 수: 100
  - 줄거리 포함 공연 수: 14
  - 줄거리 포함 비율: 14.0%
--------------------------------------------------
한국음악(국악)
  - 전체 공연 수: 22
  - 줄거리 포함 공연 수: 5
  - 줄거리 포함 비율: 22.7%
--------------------------------------------------
대중음악
  - 전체 공연 수: 100
  - 줄거리 포함 공연 수: 6
  - 줄거리 포함 비율: 6.0%
--------------------------------------------------
복합
  - 전체 공연 수: 19
  - 줄거리 포함 공연 수: 1
  - 줄거리 포함 비율: 5.3%
--------------------------------------------------
서커스/마술
  - 전체 공연 수: 100
  - 줄거리 포함 공연 수: 5
  - 줄거리 포함 비율: 5.0%
--------------------------------------------------
뮤지컬
  - 전체 공연 수: 100
  - 줄거리 포함 공연 수: 0
  - 줄거리 포함 비율: 0.0%
--------------------------------------------------


In [22]:
import os
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
from dotenv import load_dotenv
import json

# API 키 로드
load_dotenv()
KOPIS_API_KEY = os.getenv('KOPIS_API_KEY')

def debug_api_response(genre_code='CCCC', days=60):
    """KOPIS API 응답을 단계별로 검증"""
    
    # 1. 기본 정보 출력
    print("=== API 호출 정보 ===")
    today = datetime.now()
    start_date = today.strftime('%Y%m%d')
    end_date = (today + timedelta(days=days)).strftime('%Y%m%d')
    
    print(f"검색 기간: {start_date} ~ {end_date}")
    print(f"장르 코드: {genre_code}\n")
    
    # 2. 공연 목록 API 호출 검증
    print("=== 공연 목록 API 호출 ===")
    base_url = "http://www.kopis.or.kr/openApi/restful/pblprfr"
    params = {
        'service': KOPIS_API_KEY,
        'stdate': start_date,
        'eddate': end_date,
        'rows': 100,  # 테스트를 위해 100개로 제한
        'cpage': 1,
        'shcate': genre_code
    }
    
    try:
        response = requests.get(base_url, params=params)
        print(f"목록 API 상태 코드: {response.status_code}")
        
        if response.status_code == 200:
            root = ET.fromstring(response.text)
            performances = root.findall('.//db')
            print(f"검색된 공연 수: {len(performances)}\n")
            
            # 3. 각 공연별 상세 정보 검증
            print("=== 공연별 상세 정보 검증 ===")
            for idx, perf in enumerate(performances, 1):
                mt20id = perf.find('mt20id').text
                prfnm = perf.find('prfnm').text
                print(f"\n공연 {idx}) {prfnm} (ID: {mt20id})")
                
                # 상세 정보 API 호출
                detail_url = f"http://www.kopis.or.kr/openApi/restful/pblprfr/{mt20id}"
                detail_response = requests.get(detail_url, params={'service': KOPIS_API_KEY})
                print(f"상세정보 API 상태 코드: {detail_response.status_code}")
                
                if detail_response.status_code == 200:
                    detail_root = ET.fromstring(detail_response.text)
                    
                    # 주요 필드들 검증
                    fields = ['mt20id', 'prfnm', 'prfpdfrom', 'prfpdto', 'fcltynm', 'sty']
                    for field in fields:
                        elem = detail_root.find(f'.//{field}')
                        value = elem.text if elem is not None else None
                        if field == 'sty' and value:
                            value = value[:100] + '...' if len(value) > 100 else value
                        print(f"- {field}: {value}")
                
                if idx >= 10:  # 처음 10개 공연만 자세히 검증
                    remaining = len(performances) - 100
                    if remaining > 0:
                        print(f"\n(나머지 {remaining}개 공연 생략...)")
                    break
                    
            # 4. 줄거리(sty) 필드 통계
            print("\n=== 줄거리 포함 통계 ===")
            plot_count = 0
            for perf in performances:
                mt20id = perf.find('mt20id').text
                detail_url = f"http://www.kopis.or.kr/openApi/restful/pblprfr/{mt20id}"
                detail_response = requests.get(detail_url, params={'service': KOPIS_API_KEY})
                
                if detail_response.status_code == 200:
                    detail_root = ET.fromstring(detail_response.text)
                    sty = detail_root.find('.//sty')
                    if sty is not None and sty.text and sty.text.strip():
                        plot_count += 1
            
            total = len(performances)
            plot_ratio = (plot_count / total * 100) if total > 0 else 0
            print(f"전체 공연 수: {total}")
            print(f"줄거리 포함 공연 수: {plot_count}")
            print(f"줄거리 포함 비율: {plot_ratio:.1f}%")
            
        else:
            print(f"목록 API 오류 응답: {response.text}")
            
    except Exception as e:
        print(f"에러 발생: {str(e)}")

if __name__ == "__main__":
    if not KOPIS_API_KEY:
        raise ValueError("KOPIS_API_KEY가 설정되지 않았습니다.")
        
    # 뮤지컬 장르 검증
    print("뮤지컬 장르 데이터 검증\n" + "="*50)
    debug_api_response(genre_code='CCCC')

뮤지컬 장르 데이터 검증
=== API 호출 정보 ===
검색 기간: 20241230 ~ 20250228
장르 코드: CCCC

=== 공연 목록 API 호출 ===
목록 API 상태 코드: 200
검색된 공연 수: 18

=== 공연별 상세 정보 검증 ===

공연 1) 찬란한 우리 (ID: PF256353)
상세정보 API 상태 코드: 200
- mt20id: PF256353
- prfnm: 찬란한 우리
- prfpdfrom: 2025.01.23
- prfpdto: 2025.01.23
- fcltynm: 부평아트센터 (해누리극장)
- sty:  

공연 2) 해원해줄게요: REMASTER (ID: PF256296)
상세정보 API 상태 코드: 200
- mt20id: PF256296
- prfnm: 해원해줄게요: REMASTER
- prfpdfrom: 2025.01.24
- prfpdto: 2025.01.26
- fcltynm: 대학로예술극장 (소극장)
- sty: [공연소개]

간절히 바라고 빌던 그 마음, 해원
(‘해원(解寃)’은 원통한 마음을 풀어주거나 바라는 것을 이루고자 신, 또는 조상에게 비는 한국의 전통적인 문화를 통칭한다....

공연 3) 야금밴드 콘서트 [충남] (ID: PF256289)
상세정보 API 상태 코드: 200
- mt20id: PF256289
- prfnm: 야금밴드 콘서트 [충남]
- prfpdfrom: 2025.01.26
- prfpdto: 2025.01.26
- fcltynm: 천안예술의전당 (소공연장)
- sty:  

공연 4) 가락프로젝트 다섯 번째 시리즈, 무명씨의 아홉 짐 (ID: PF256288)
상세정보 API 상태 코드: 200
- mt20id: PF256288
- prfnm: 가락프로젝트 다섯 번째 시리즈, 무명씨의 아홉 짐
- prfpdfrom: 2024.12.30
- prfpdto: 2024.12.31
- fcltynm: 문래예술공장 (박스씨어터)
- sty:  

공연 5) 경셩유행가: 판을 나온 

## 3. 공연 데이터 수집 함수 구현

In [3]:
def get_performance_list(genre_code=None):
    """장르별 공연 목록을 조회하는 함수"""
    today = datetime.now()
    start_date = today.strftime('%Y%m%d')
    end_date = (today + timedelta(days=90)).strftime('%Y%m%d')
    
    base_url = "http://www.kopis.or.kr/openApi/restful/pblprfr"
    params = {
        'service': KOPIS_API_KEY,
        'stdate': start_date,
        'eddate': end_date,
        'rows': 100,
        'cpage': 1
    }
    
    if genre_code:
        params['shcate'] = genre_code
    
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        raise Exception(f"API 요청 실패: {response.status_code}")
        
    return response.text

def get_performance_detail(mt20id):
    """공연 상세 정보를 조회하는 함수"""
    detail_url = f"http://www.kopis.or.kr/openApi/restful/pblprfr/{mt20id}"
    response = requests.get(detail_url, params={'service': KOPIS_API_KEY})
    
    if response.status_code != 200:
        return ""
        
    root = ET.fromstring(response.text)
    sty = root.find('.//sty')
    return sty.text if sty is not None and sty.text else ""

In [11]:
get_performance_list('GGGA')

'<?xml version="1.0" encoding="UTF-8"?>\n<dbs>\n    <db>\n        <mt20id>PF256377</mt20id>\n        <prfnm>피터팬 [김해]</prfnm>\n        <prfpdfrom>2025.02.08</prfpdfrom>\n        <prfpdto>2025.02.08</prfpdto>\n        <fcltynm>김해 장유문화센터</fcltynm>\n        <poster>http://www.kopis.or.kr/upload/pfmPoster/PF_PF256377_241227_135838.png</poster>\n        <area>경상남도</area>\n        <genrenm>뮤지컬</genrenm>\n        <openrun>N</openrun>\n        <prfstate>공연예정</prfstate>\n    </db>\n    <db>\n        <mt20id>PF256362</mt20id>\n        <prfnm>신데렐라 [부산]</prfnm>\n        <prfpdfrom>2025.01.11</prfpdfrom>\n        <prfpdto>2025.01.11</prfpdto>\n        <fcltynm>영도문화예술회관</fcltynm>\n        <poster>http://www.kopis.or.kr/upload/pfmPoster/PF_PF256362_241227_130208.gif</poster>\n        <area>부산광역시</area>\n        <genrenm>뮤지컬</genrenm>\n        <openrun>N</openrun>\n        <prfstate>공연예정</prfstate>\n    </db>\n    <db>\n        <mt20id>PF256361</mt20id>\n        <prfnm>반쪽이전 [인천]</prfnm>\n        <prfpd

## 4. 추천 시스템 구현

In [4]:
class PerformanceRecommender:
    def __init__(self):
        self.performances = pd.DataFrame()
        self.tfidf = TfidfVectorizer(stop_words='english')
        
    def collect_performances(self):
        """모든 장르의 공연 정보 수집"""
        all_performances = []
        
        for genre_code in GENRE_CODES.keys():
            try:
                xml_data = get_performance_list(genre_code)
                root = ET.fromstring(xml_data)
                
                for item in root.findall('.//db'):
                    perf = {
                        'mt20id': item.find('mt20id').text,
                        'prfnm': item.find('prfnm').text,
                        'genrenm': item.find('genrenm').text,
                        'prfpdfrom': item.find('prfpdfrom').text,
                        'prfpdto': item.find('prfpdto').text,
                        'fcltynm': item.find('fcltynm').text
                    }
                    all_performances.append(perf)
                    
            except Exception as e:
                print(f"장르 {genre_code} 데이터 수집 중 오류: {e}")
                
        self.performances = pd.DataFrame(all_performances)
        self.performances['plot'] = self.performances['mt20id'].apply(get_performance_detail)
        
        # 줄거리가 없는 공연 제외
        self.performances = self.performances[self.performances['plot'].str.len() > 0].reset_index(drop=True)
        
    def get_recommendations(self, input_text, n_recommendations=5):
        """입력된 텍스트와 유사한 줄거리를 가진 공연 추천"""
        if self.performances.empty:
            raise ValueError("공연 데이터가 수집되지 않았습니다.")
            
        # TF-IDF 행렬 생성
        plot_list = self.performances['plot'].tolist()
        plot_list.append(input_text)
        tfidf_matrix = self.tfidf.fit_transform(plot_list)
        
        # 코사인 유사도 계산
        cosine_sim = cosine_similarity(tfidf_matrix[-1:], tfidf_matrix[:-1])[0]
        
        # 유사도가 높은 순으로 정렬
        sim_scores = list(enumerate(cosine_sim))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # 추천 결과 반환
        recommended_indices = [i[0] for i in sim_scores[:n_recommendations]]
        recommendations = self.performances.iloc[recommended_indices].copy()
        recommendations['similarity_score'] = [score for _, score in sim_scores[:n_recommendations]]
        
        return recommendations[['prfnm', 'genrenm', 'fcltynm', 'similarity_score']]

# 추천 시스템 테스트
recommender = PerformanceRecommender()
recommender.collect_performances()

# 테스트 입력
test_input = """
사랑과 이별, 그리고 용서에 대한 이야기.
한 젊은이가 자신의 과거와 마주하며 성장하는 과정을 그린 감동적인 드라마.
"""

recommendations = recommender.get_recommendations(test_input)
print("추천된 공연 목록:")
display(recommendations)

추천된 공연 목록:


,prfnm,genrenm,fcltynm,similarity_score
38,내 무덤에 너를 묻고,연극,아르코예술극장,0.051345
13,게릴라 씨어터,연극,극장 동국,0.037048
429,LUKAS 사유의 시간,서커스/마술,문엔트리,0.034150
89,시인의 레퀴엠 [수원],연극,소극장 울림터(극단 메카네),0.021678
297,"보수동쿨러 마지막 공연, 커다란 환호를 보내 [부산]",대중음악,오방가르드,0.019827


## 5. 추천 결과 분석

In [13]:
def analyze_recommendations(recommendations, original_data):
    """추천 결과 분석"""
    print("=== 추천 결과 분석 ===")
    
    # 장르 분포
    print("\n장르 분포:")
    display(recommendations['genrenm'].value_counts())
    
    # 유사도 점수 통계
    print("\n유사도 점수 통계:")
    print(recommendations['similarity_score'].describe())
    
    # 전체 데이터와 비교
    print("\n전체 데이터 대비 장르 비율:")
    total_genre_dist = original_data['genrenm'].value_counts(normalize=True)
    rec_genre_dist = recommendations['genrenm'].value_counts(normalize=True)
    comparison = pd.DataFrame({
        '추천 결과 비율': rec_genre_dist,
        '전체 데이터 비율': total_genre_dist
    }).fillna(0)
    display(comparison)

# 분석 실행
analyze_recommendations(recommendations, recommender.performances)

=== 추천 결과 분석 ===

장르 분포:


genrenm
연극        3
서커스/마술    1
대중음악      1
Name: count, dtype: int64


유사도 점수 통계:
count    5.000000
mean     0.032810
std      0.012804
min      0.019827
25%      0.021678
50%      0.034150
75%      0.037048
max      0.051345
Name: similarity_score, dtype: float64

전체 데이터 대비 장르 비율:


,추천 결과 비율,전체 데이터 비율
genrenm,,
대중음악,0.2,0.180505
무용,0.0,0.023466
뮤지컬,0.0,0.180505
복합,0.0,0.034296
서양음악(클래식),0.0,0.180505
서커스/마술,0.2,0.180505
연극,0.6,0.180505
한국음악(국악),0.0,0.039711
